In [49]:
"""
Downloading data
"""
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import numpy as np
import time

years = range(1, 26)
seasons = [f"20{i:02d}-{(i+1)%100:02d}" for i in years]

to_merge = []
for s in seasons:
    time.sleep(1)
    finder = leaguegamefinder.LeagueGameFinder(
        season_nullable=s,
        season_type_nullable="Regular Season",
        league_id_nullable='00'
    )

    game_df = finder.get_data_frames()[0]
    to_merge.append(game_df)

games_df = pd.concat(to_merge)

In [50]:
"""Time-aware splitter that purges a gap between train and validation groups."""
import numpy as np

class PurgedGroupTimeSeriesSplit:
    """Drop-in replacement for TimeSeriesSplit with group-aware purging."""
    def __init__(self, n_splits=5, *, group_gap=1, max_train_group_size=None, test_group_size=None):
        if n_splits < 2:
            raise ValueError("n_splits must be at least 2")
        self.n_splits = n_splits
        self.group_gap = int(group_gap)
        self.max_train_group_size = max_train_group_size
        self.test_group_size = test_group_size

    def split(self, X, y=None, groups=None):
        if groups is None:
            raise ValueError("PurgedGroupTimeSeriesSplit requires 'groups' (e.g. normalized dates)")
        groups = np.asarray(groups)
        unique, first_idx = np.unique(groups, return_index=True)
        order = np.argsort(first_idx)
        unique_groups = unique[order]
        group_to_pos = {g: i for i, g in enumerate(unique_groups)}
        group_ids = np.fromiter((group_to_pos[g] for g in groups), dtype=int, count=len(groups))
        n_groups = len(unique_groups)
        if self.n_splits >= n_groups:
            raise ValueError("Too few unique groups for the requested number of splits")

        test_size = self.test_group_size or max(1, n_groups // (self.n_splits + 1))

        for fold in range(self.n_splits):
            test_start = n_groups - (self.n_splits - fold) * test_size
            test_end = min(test_start + test_size, n_groups)
            if test_start <= 0:
                continue
            train_end = max(0, test_start - self.group_gap)
            if self.max_train_group_size is not None:
                train_start = max(0, train_end - int(self.max_train_group_size))
            else:
                train_start = 0

            train_mask = (group_ids >= train_start) & (group_ids < train_end)
            val_mask = (group_ids >= test_start) & (group_ids < test_end)
            train_idx = np.flatnonzero(train_mask)
            val_idx = np.flatnonzero(val_mask)
            if len(train_idx) == 0 or len(val_idx) == 0:
                continue
            yield train_idx, val_idx

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits


In [51]:
"""
Merging games into single rows
"""

def merge_games(df):
    df['HOME'] = df['MATCHUP'].str.contains('vs.') 
    home_df = df[df['HOME']].copy()
    away_df = df[~df['HOME']].copy()

    merged = home_df.merge(
        away_df,
        on='GAME_ID',
        suffixes=('_home', '_away')
    )
    merged = merged.rename(columns={
        'SEASON_ID_home' : 'sid',
        'GAME_DATE_home': 'date',
        'TEAM_ABBREVIATION_home': 'team_home',
        'TEAM_ABBREVIATION_away': 'team_away',
        'PTS_home': 'score_home',
        'PTS_away': 'score_away'
    })

    merged["win_home"] = (merged["score_home"] > merged["score_away"]).astype(int)
    merged['margin'] = (merged['score_home'] - merged['score_away'])

    merged['date'] = pd.to_datetime(merged['date'])
    merged = merged.sort_values('date').reset_index(drop=True)
    drop_cols = [
    'TEAM_ID_home', 'TEAM_ID_away', 'TEAM_NAME_home', 'TEAM_NAME_away',
    'MATCHUP_home', 'MATCHUP_away', 'GAME_ID',
    'WL_home', 'WL_away', 'HOME_home', 'HOME_away']
    merged.drop(drop_cols, axis=1, inplace=True)
    return merged

games_df = merge_games(games_df)
print(games_df.columns)
games_df


Index(['sid', 'team_home', 'date', 'MIN_home', 'score_home', 'FGM_home',
       'FGA_home', 'FG_PCT_home', 'FG3M_home', 'FG3A_home', 'FG3_PCT_home',
       'FTM_home', 'FTA_home', 'FT_PCT_home', 'OREB_home', 'DREB_home',
       'REB_home', 'AST_home', 'STL_home', 'BLK_home', 'TOV_home', 'PF_home',
       'PLUS_MINUS_home', 'SEASON_ID_away', 'team_away', 'GAME_DATE_away',
       'MIN_away', 'score_away', 'FGM_away', 'FGA_away', 'FG_PCT_away',
       'FG3M_away', 'FG3A_away', 'FG3_PCT_away', 'FTM_away', 'FTA_away',
       'FT_PCT_away', 'OREB_away', 'DREB_away', 'REB_away', 'AST_away',
       'STL_away', 'BLK_away', 'TOV_away', 'PF_away', 'PLUS_MINUS_away',
       'win_home', 'margin'],
      dtype='object')


,sid,team_home,date,MIN_home,score_home,FGM_home,FGA_home,FG_PCT_home,FG3M_home,FG3A_home,...,DREB_away,REB_away,AST_away,STL_away,BLK_away,TOV_away,PF_away,PLUS_MINUS_away,win_home,margin
0,22001,NYK,2001-10-30,239,93,32,75,0.427,6,20,...,30,39,23,11,4,14,27,-2.0,1,2
1,22001,MIN,2001-10-30,239,83,34,81,0.420,3,9,...,26,41,16,4,6,18,20,-9.0,1,9
2,22001,UTA,2001-10-30,264,112,43,81,0.531,5,16,...,21,31,25,11,3,13,28,7.0,0,-7
3,22001,CLE,2001-10-30,240,89,39,90,0.433,3,17,...,38,48,24,6,2,14,14,19.0,0,-19
4,22001,SAS,2001-10-30,240,109,37,80,0.463,11,24,...,31,42,9,11,3,18,31,-11.0,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28933,22025,ATL,2025-11-04,240,127,40,72,0.556,13,30,...,27,41,26,9,1,17,30,-15.0,1,15
28934,22025,GSW,2025-11-04,240,118,41,83,0.494,19,42,...,30,46,25,10,5,17,17,-11.0,1,11
28935,22025,TOR,2025-11-04,241,128,44,91,0.484,17,38,...,32,41,26,5,4,12,26,-28.0,1,28
28936,22025,NOP,2025-11-04,240,116,43,90,0.478,17,38,...,34,49,20,8,6,19,23,-4.0,1,4


In [52]:
import numpy as np
import pandas as pd
import math
from scipy.optimize import minimize

EPS = 1e-12

games_sorted = games_df.sort_values("date").reset_index(drop=False)
cut1 = games_sorted["date"].quantile(0.8)
tr_idx = games_sorted.loc[games_sorted["date"] < cut1, "index"].to_numpy()

p_home = (games_df["win_home"] == 1).mean()
HCA = 400 * math.log10(p_home / (1 - p_home))

def _elo_logloss_for_params(games_df, K, HCA, alpha, scale=400.0):
    """
    Compute total negative log-likelihood for given Elo params on a chronologically
    ordered DataFrame with columns: sid, date, team_home, team_away, score_home, score_away.
    """
    teams = pd.unique(games_df[['team_home','team_away']].values.ravel())
    elo = {t: 1500.0 for t in teams}
    prev_sid = object()
    nll = 0.0

    for _, r in games_df.iterrows():
        # season reset (shrink to mean)
        if r['sid'] != prev_sid:
            for t in elo:
                elo[t] = alpha * elo[t] + (1.0 - alpha) * 1500.0

        Ra, Rb = elo[r.team_home], elo[r.team_away]

        # probability of home win using learned scale (400 by default)
        # elo_diff = Ra - Rb + HCA
        Ea = 1.0 / (1.0 + 10.0 ** ((Rb - (Ra + HCA)) / scale))

        y = 1.0 if r.score_home > r.score_away else 0.0
        nll += -(y * np.log(Ea + EPS) + (1 - y) * np.log(1 - Ea + EPS))

        # update ratings with margin-of-victory multiplier
        margin = abs(r.score_home - r.score_away)
        mult = np.log1p(margin) * 2.2 / ((Ra+HCA - Rb) * 0.001 + 2.2)
        delta = K * mult * (y - Ea)

        elo[r.team_home] = Ra + delta
        elo[r.team_away] = Rb - delta
        prev_sid = r['sid']

    return nll

def fit_elo_params(games_train, K0=20.0, HCA0=60.0, alpha0=0.55, scale0=400.0):
    """
    Learn K, HCA, alpha (season reset), and scale by ML on the TRAIN window only.
    games_train must be chronologically sorted.
    """
    bounds = [(5.0, 80.0),     # K
              (0.0, 120.0),    # HCA (Elo points)
              (0.20, 0.95),    # alpha (keep some shrink)
              (250.0, 550.0)]  # scale in the Elo->prob sigmoid (400 standard)

    x0 = np.array([K0, HCA0, alpha0, scale0], dtype=float)

    def objective(x):
        K, HCA, alpha, scale = x
        return _elo_logloss_for_params(games_train, K, HCA, alpha, scale)

    res = minimize(objective, x0=x0, method="L-BFGS-B", bounds=bounds)
    K, HCA, alpha, scale = res.x
    return float(K), float(HCA), float(alpha), float(scale)

# build the TRAIN slice and ensure chronological order
train_df = games_df.loc[tr_idx].sort_values("date").reset_index(drop=True)

# a reasonable HCA starting guess from your earlier empirical calc
K_star, HCA_star, alpha_star, scale_star = fit_elo_params(
    train_df, K0=20.0, HCA0=HCA, alpha0=0.55, scale0=400.0
)
print("Learned params:", K_star, HCA_star, alpha_star, scale_star)

KeyboardInterrupt: 

In [ ]:
"""
Updating Elos
"""


p_home = (games_df["win_home"] == 1).mean()
HCA = 400 * math.log10(p_home / (1 - p_home))
K = 20


def update_elo(games_df, K=K, HCA=HCA, alpha=0.55, scale=400):
    elo = {team: 1500 for team in pd.unique(games_df[['team_home', 'team_away']].values.ravel())}
    records = []

    prev_sid = object() 
    for idx, row in games_df.iterrows():
        if row['sid'] != prev_sid:
            for team, rating in elo.items():
                elo[team] = alpha * rating + (1-alpha)* 1500

        home, away = row['team_home'], row['team_away']
        score_home, score_away = row['score_home'], row['score_away']
        win_home = 1 if score_home > score_away else 0

        Ra, Rb = elo[home], elo[away]
        Ea = 1 / (1 + 10 ** ((Rb - (Ra + HCA)) / scale))
        margin = abs(score_home - score_away)
        mult = np.log1p(margin) * 2.2 / ((Ra+HCA - Rb) * 0.001 + 2.2)
        delta = K * mult * (win_home - Ea)

        elo[home] += delta
        elo[away] -= delta

        games_df.loc[idx, 'elo_pre_home'] = Ra
        games_df.loc[idx, 'elo_pre_away'] = Rb
        games_df.loc[idx, 'elo_post_home'] = elo[home]
        games_df.loc[idx, 'elo_post_away'] = elo[away]

        prev_sid = row['sid']

    games_df['elo_diff'] = games_df['elo_pre_home'] - games_df['elo_pre_away'] + HCA

    return elo

elos = update_elo(games_df, K_star, HCA_star, alpha_star, scale_star)
games_df.reset_index(drop=True, inplace=True)
games_df

,sid,team_home,date,MIN_home,score_home,FGM_home,FGA_home,FG_PCT_home,FG3M_home,FG3A_home,...,TOV_away,PF_away,PLUS_MINUS_away,win_home,margin,elo_pre_home,elo_pre_away,elo_post_home,elo_post_away,elo_diff
0,22001,NYK,2001-10-30,239,93,32,75,0.427,6,20,...,14,27,-2.0,1,2,1500.000000,1500.000000,1504.035469,1495.964531,75.917651
1,22001,MIN,2001-10-30,239,83,34,81,0.420,3,9,...,18,20,-9.0,1,9,1500.000000,1500.000000,1508.457953,1491.542047,75.917651
2,22001,UTA,2001-10-30,264,112,43,81,0.531,5,16,...,13,28,7.0,0,-7,1500.000000,1500.000000,1488.175682,1511.824318,75.917651
3,22001,CLE,2001-10-30,240,89,39,90,0.433,3,17,...,14,14,19.0,0,-19,1500.000000,1500.000000,1482.965383,1517.034617,75.917651
4,22001,SAS,2001-10-30,240,109,37,80,0.463,11,24,...,18,31,-11.0,1,11,1500.000000,1500.000000,1509.127665,1490.872335,75.917651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28933,22025,TOR,2025-11-04,241,128,44,91,0.484,17,38,...,12,26,-28.0,1,28,1449.149385,1577.107186,1468.328494,1557.928077,-52.040150
28934,22025,CHI,2025-11-04,240,113,42,89,0.472,13,34,...,16,21,-2.0,1,2,1542.823145,1427.068849,1545.260822,1424.631172,191.671948
28935,22025,LAC,2025-11-04,242,107,33,80,0.413,18,43,...,10,23,19.0,0,-19,1563.195444,1691.366159,1550.556904,1704.004699,-52.253064
28936,22025,GSW,2025-11-04,240,118,41,83,0.494,19,42,...,17,17,-11.0,1,11,1566.505744,1464.445835,1572.385845,1458.565734,177.977560


In [ ]:
import pandas as pd
import numpy as np

WIN_WINDOW = 10

def add_features(games: pd.DataFrame, win_window: int = WIN_WINDOW, fill_rest: int = 7,
                 roll_windows=(3, 5)) -> pd.DataFrame:
    df = games.copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    if "sid" not in df.columns:
        raise ValueError("Input must include 'sid' (season id).")
    df["sid"] = pd.to_numeric(df["sid"], errors="coerce")

    if "win_home" not in df.columns:
        if {"score_home", "score_away"}.issubset(df.columns):
            df["win_home"] = (df["score_home"] > df["score_away"]).astype(int)
        else:
            raise ValueError("Need 'win_home' or both 'score_home' and 'score_away'.")

    # stable row id
    df = df.reset_index(drop=False).rename(columns={"index": "row_id"})

    # ------- Box-score efficiency (same-game) -------
    df["off_eff_home"] = df["score_home"] / df["FGA_home"]
    df["off_eff_away"] = df["score_away"] / df["FGA_away"]
    df['def_eff_home'] = (df['BLK_home'] + df['STL_home'] - df['PF_home']) / df['score_away']
    df['def_eff_away'] = (df['BLK_away'] + df['STL_away'] - df['PF_away']) / df['score_home']

    df['poss_home'] = 0.96 * (df['FGA_home'] + 0.44*df['FTA_home'] - df['OREB_home'] + df['TOV_home'])
    df['poss_away'] = 0.96 * (df['FGA_away'] + 0.44*df['FTA_away'] - df['OREB_away'] + df['TOV_away'])
    df['pace_home'] = (df['poss_home'] + df['poss_away']) / (2 * df['MIN_home'])
    df['pace_away'] = (df['poss_home'] + df['poss_away']) / (2 * df['MIN_away'])

    df['off_rtg_home'] = df['score_home'] / df['poss_home'] * 100
    df['def_rtg_home'] = df['score_away'] / df['poss_away'] * 100
    df['off_rtg_away'] = df['score_away'] / df['poss_away'] * 100
    df['def_rtg_away'] = df['score_home'] / df['poss_home'] * 100

    df['fgp_home'] = df['FGM_home'] / df['FGA_home']
    df['fgp_away'] = df['FGM_away'] / df['FGA_away']
    df['tpp_home'] = df['FG3M_home'] / df['FG3A_home']
    df['tpp_away'] = df['FG3M_away'] / df['FG3A_away']

    df["efg_home"]     = (df["FGM_home"] + 0.5 * df["FG3M_home"]) / df["FGA_home"]
    df["efg_away"]     = (df["FGM_away"] + 0.5 * df["FG3M_away"]) / df["FGA_away"]

    df["tov_rate_home"] = df["TOV_home"] * 100 / (df["FGA_home"] + 0.44 * df["FTA_home"] + df['AST_home'] + df["TOV_home"])
    df["tov_rate_away"] = df["TOV_away"] * 100 / (df["FGA_away"] + 0.44 * df["FTA_away"] + df['AST_away'] + df["TOV_away"])
    df['ast_tov_ratio_home'] = df['AST_home'] / df['TOV_home']
    df['ast_tov_ratio_away'] = df['AST_away'] / df['TOV_away']

    df["reb_rate_home"] = df["REB_home"] / (df["REB_home"] + df["REB_away"])
    df["reb_rate_away"] = 1 - df["reb_rate_home"]

    df['orbp_home'] = df['OREB_home'] / (df['OREB_home'] + df['DREB_away']) 
    df['orbp_away'] = df['OREB_away'] / (df['OREB_away'] + df['DREB_home']) 
    df['drbp_home'] = df['DREB_home'] / (df['DREB_home'] + df['OREB_away'])
    df['drbp_away'] = df['DREB_away'] / (df['DREB_away'] + df['OREB_home'])

    df["ft_eff_home"]   = df["FTM_home"] / df["FTA_home"].replace(0, np.nan)
    df["ft_eff_away"]   = df["FTM_away"] / df["FTA_away"].replace(0, np.nan)

    df['ff_home'] = 0.4*df['efg_home'] + 0.25*df['tov_rate_home'] + 0.2 * (df['orbp_home'] + df['drbp_home']) / 2 + 0.15*df['ft_eff_home']
    df['ff_away'] = 0.4*df['efg_away'] + 0.25*df['tov_rate_away'] + 0.2 * (df['orbp_away'] + df['drbp_away']) / 2 + 0.15*df['ft_eff_away']


    # ------- Per-team logs (vectorized) -------
    home = df[[
    "sid","date","team_home","score_home","score_away","win_home","row_id",
    "off_rtg_home","def_rtg_home"
    ]].rename(columns={
        "team_home":"team",
        "score_home":"pf",
        "score_away":"pa",
        "win_home":"win",
        "off_rtg_home":"off_rtg",
        "def_rtg_home":"def_rtg",
    })
    home["is_home"] = 1

    away = df[[
        "sid","date","team_away","score_home","score_away","win_home","row_id",
        "off_rtg_away","def_rtg_away"
    ]].rename(columns={
        "team_away":"team",
        "score_home":"pa",
        "score_away":"pf",
        "win_home":"win",
        "off_rtg_away":"off_rtg",
        "def_rtg_away":"def_rtg",
    })
    away["is_home"] = 0
    away["win"] = 1 - away["win"]

    logs = pd.concat([home, away], ignore_index=True)
    logs = logs.sort_values(["team","sid","date","row_id"], kind="mergesort").reset_index(drop=True)
    logs["pt_diff"] = (logs["pf"] - logs["pa"]).astype(float)
    g = logs.groupby(["team","sid"], sort=False)

    # ------- Season-to-date BEFORE this game -------
    logs["spdiff"] = (g["pt_diff"].cumsum() - logs["pt_diff"]).astype(float)
    gp_incl   = g.cumcount() + 1
    gp_prev   = gp_incl - 1
    pf_cum_in = g["pf"].cumsum()
    pa_cum_in = g["pa"].cumsum()
    pf_pre    = pf_cum_in - logs["pf"]
    pa_pre    = pa_cum_in - logs["pa"]
    logs["off_eff"] = np.where(gp_prev > 0, pf_pre / gp_prev, np.nan)
    logs["def_eff"] = np.where(gp_prev > 0, pa_pre / gp_prev, np.nan)

    logs["home_game"]  = (logs["is_home"] == 1).astype(int)
    logs["away_game"]  = (logs["is_home"] == 0).astype(int)
    logs["home_win"]   = logs["win"] * logs["home_game"]
    logs["away_win"]   = logs["win"] * logs["away_game"]
    logs["home_pdiff"] = logs["pt_diff"] * logs["home_game"]
    logs["away_pdiff"] = logs["pt_diff"] * logs["away_game"]

    h_games_in = g["home_game"].cumsum()
    a_games_in = g["away_game"].cumsum()
    h_wins_in  = g["home_win"].cumsum()
    a_wins_in  = g["away_win"].cumsum()
    h_pd_in    = g["home_pdiff"].cumsum()
    a_pd_in    = g["away_pdiff"].cumsum()

    h_games = h_games_in - logs["home_game"]
    a_games = a_games_in - logs["away_game"]
    h_wins  = h_wins_in  - logs["home_win"]
    a_wins  = a_wins_in  - logs["away_win"]
    h_pd    = h_pd_in    - logs["home_pdiff"]
    a_pd    = a_pd_in    - logs["away_pdiff"]

    logs["home_win_pct"]    = np.where(h_games > 0, h_wins / h_games, np.nan)
    logs["away_win_pct"]    = np.where(a_games > 0, a_wins / a_games, np.nan)
    logs["home_point_diff"] = np.where(h_games > 0, h_pd / h_games, np.nan)
    logs["away_point_diff"] = np.where(a_games > 0, a_pd / a_games, np.nan)

    # ------- Rest / rw_pct / streak -------
    logs["prev_date"] = g["date"].shift(1)
    logs["rest_days"] = ((logs["date"] - logs["prev_date"]).dt.days - 1).clip(lower=0)
    logs["prev_win"]  = g["win"].shift(1)
    logs["rw_pct"]    = (g["prev_win"].rolling(win_window, min_periods=1).mean()
                         .reset_index(level=[0,1], drop=True))

    logs["prev_sign"] = logs["prev_win"].map({1: 1, 0: -1})
    def _streak_from_prev(gr):
        cur, out = 0, []
        for s in gr["prev_sign"]:
            if pd.isna(s):
                cur = 0
            else:
                s = int(s)
                cur = (cur + 1 if cur > 0 else 1) if s > 0 else (cur - 1 if cur < 0 else -1)
            out.append(cur)
        gr = gr.copy()
        gr["streak"] = out
        return gr
    logs = logs.groupby(["team","sid"], group_keys=False).apply(_streak_from_prev)

    # ------- Elo PRE rolling (uses df['elo_pre_home'], df['elo_pre_away'] if present) -------
    if {"elo_pre_home", "elo_pre_away"}.issubset(df.columns):
        logs["elo_pre"] = np.where(
            logs["is_home"] == 1,
            df.loc[logs["row_id"].values, "elo_pre_home"].values,
            df.loc[logs["row_id"].values, "elo_pre_away"].values
        ).astype(float)
        logs["opp_elo_pre"] = np.where(
                logs["is_home"] == 1,
                df.loc[logs["row_id"].values, "elo_pre_away"].values,
                df.loc[logs["row_id"].values, "elo_pre_home"].values).astype(float)
        logs["elo_vs_opp"] = logs["elo_pre"] - logs["opp_elo_pre"]
    

        def _elo_roll(gr, w=5):
            e = gr["elo_pre"].astype(float)
            gr = gr.copy()
            gr["elo_pre_ma5"]       = e.rolling(w, min_periods=3).mean()
            gr["elo_pre_slope5"]    = e.rolling(w, min_periods=3).apply(
                lambda x: np.polyfit(np.arange(len(x)), x, 1)[0], raw=False
            )
            de = e.diff()
            gr["elo_pre_diff_ma5"]  = de.rolling(w, min_periods=3).mean()
            gr["elo_pre_diff_std5"] = de.rolling(w, min_periods=3).std()
            return gr

        logs = logs.groupby(["team","sid"], group_keys=False).apply(_elo_roll)
        logs["elo_vs_opp_roll5"] = logs.groupby(["team","sid"])['elo_vs_opp'].transform(lambda s: s.shift(1).rolling(5, min_periods=1).mean())
    
    else:
        logs["elo_pre_ma5"] = np.nan
        logs["elo_pre_slope5"] = np.nan
        logs["elo_pre_diff_ma5"] = np.nan
        logs["elo_pre_diff_std5"] = np.nan
        logs["elo_vs_opp"] = np.nan
        logs["elo_vs_opp_roll5"] = np.nan
    

    # ------- Spread back to game rows -------
    h = logs.rename(columns={
        "team":"team_home",
        "rest_days":"rest_h","rw_pct":"rw_pct_h","streak":"streak_h",
        "spdiff":"spdiff_h",
        "off_eff":"off_eff_h","def_eff":"def_eff_h",
        "home_win_pct":"home_win_pct_h",
        "home_point_diff":"home_point_diff_h",
        "elo_pre_ma5":"elo_pre_ma5_h",
        "elo_pre_slope5":"elo_pre_slope5_h",
        "elo_pre_diff_ma5":"elo_momentum_h",
        'elo_vs_opp_roll5' : 'elo_vs_opp_roll5_h',
        "elo_pre_diff_std5":"elo_volatility_h",
        'off_rtg' : 'off_rtg_h',
        'def_rtg' : 'def_rtg_h',
    })[["sid","date","team_home",
        "rest_h","rw_pct_h","streak_h","spdiff_h",
        "off_eff_h","def_eff_h",
        "home_win_pct_h","home_point_diff_h", 'elo_vs_opp_roll5_h',
        "elo_pre_ma5_h","elo_pre_slope5_h","elo_momentum_h","elo_volatility_h", 'def_rtg_h', 'off_rtg_h']]

    a = logs.rename(columns={
        "team":"team_away",
        "rest_days":"rest_a","rw_pct":"rw_pct_a","streak":"streak_a",
        "spdiff":"spdiff_a",
        "off_eff":"off_eff_a","def_eff":"def_eff_a",
        "away_win_pct":"away_win_pct_a",
        "away_point_diff":"away_point_diff_a",
        "elo_pre_ma5":"elo_pre_ma5_a",
        "elo_pre_slope5":"elo_pre_slope5_a",
        "elo_pre_diff_ma5":"elo_momentum_a",
        'elo_vs_opp_roll5' : 'elo_vs_opp_roll5_a',
        "elo_pre_diff_std5":"elo_volatility_a",
        'off_rtg' : 'off_rtg_a',
        'def_rtg' : 'def_rtg_a',
    })[["sid","date","team_away",
        "rest_a","rw_pct_a","streak_a","spdiff_a",
        "off_eff_a","def_eff_a",
        "away_win_pct_a","away_point_diff_a", 'elo_vs_opp_roll5_a',
        "elo_pre_ma5_a","elo_pre_slope5_a","elo_momentum_a","elo_volatility_a", 'def_rtg_a', 'off_rtg_a']]

    out = (df.drop(columns="row_id")
             .merge(h, on=["sid","date","team_home"], how="left")
             .merge(a, on=["sid","date","team_away"], how="left"))

    # ------- Box-score prev & rolling (3,5) for ALL numeric _home/_away bases -------
    numeric_cols = out.select_dtypes(include=[np.number]).columns.tolist()
    home_bases = {c[:-5] for c in out.columns if c.endswith("_home") and c in numeric_cols}
    away_bases = {c[:-5] for c in out.columns if c.endswith("_away") and c in numeric_cols}
    bases = sorted(home_bases & away_bases)

    H = out[["date","team_home"] + [f"{b}_home" for b in bases]].rename(
        columns={"team_home":"team", **{f"{b}_home": b for b in bases}}
    )
    A = out[["date","team_away"] + [f"{b}_away" for b in bases]].rename(
        columns={"team_away":"team", **{f"{b}_away": b for b in bases}}
    )
    long = pd.concat([H, A], ignore_index=True).sort_values(["team","date"])

    prev = (long.groupby("team", group_keys=False)
                 .apply(lambda g: g.assign(**{f"{b}_prev": g[b].shift(1) for b in bases}))
                 [["team","date"] + [f"{b}_prev" for b in bases]])

    rolls = []
    for w in roll_windows:
        r = (long.groupby("team", group_keys=False)
                  .apply(lambda g: g.assign(**{
                      f"{b}_roll{w}": g[b].ewm(w, min_periods=1).mean().shift(1)
                      for b in bases
                  }))
                  [["team","date"] + [f"{b}_roll{w}" for b in bases]])
        rolls.append(r)

    long_roll = prev
    for r in rolls:
        long_roll = long_roll.merge(r, on=["team","date"], how="left")

    home_feat = long_roll.rename(columns={
        **{f"{b}_prev": f"{b}_home_prev" for b in bases},
        **{c: c.replace("_roll", "_home_roll") for c in long_roll.columns if "_roll" in c}
    })
    away_feat = long_roll.rename(columns={
        **{f"{b}_prev": f"{b}_away_prev" for b in bases},
        **{c: c.replace("_roll", "_away_roll") for c in long_roll.columns if "_roll" in c}
    })

    out = out.merge(home_feat, left_on=["team_home","date"], right_on=["team","date"], how="left").drop(columns=["team"])
    out = out.merge(away_feat, left_on=["team_away","date"], right_on=["team","date"], how="left").drop(columns=["team"])

    # diffs for prev and rolling
    for b in bases:
        out[f"{b}_prev_diff"] = out[f"{b}_home_prev"] - out[f"{b}_away_prev"]
    for w in roll_windows:
        for b in bases:
            out[f"{b}_roll{w}_diff"] = out[f"{b}_home_roll{w}"] - out[f"{b}_away_roll{w}"]

    # ------- Fills -------
    out["rest_h"]   = out["rest_h"].fillna(fill_rest)
    out["rest_a"]   = out["rest_a"].fillna(fill_rest)
    out["rw_pct_h"] = out["rw_pct_h"].fillna(0.5)
    out["rw_pct_a"] = out["rw_pct_a"].fillna(0.5)
    out["streak_h"] = out["streak_h"].fillna(0).astype(int)
    out["streak_a"] = out["streak_a"].fillna(0).astype(int)
    out["home_win_pct_h"]    = out["home_win_pct_h"].fillna(0.5)
    out["away_win_pct_a"]    = out["away_win_pct_a"].fillna(0.5)
    out["home_point_diff_h"] = out["home_point_diff_h"].fillna(0.0)
    out["away_point_diff_a"] = out["away_point_diff_a"].fillna(0.0)
    out["off_eff_h"] = out["off_eff_h"].fillna(0.0)
    out["off_eff_a"] = out["off_eff_a"].fillna(0.0)
    out["def_eff_h"] = out["def_eff_h"].fillna(0.0)
    out["def_eff_a"] = out["def_eff_a"].fillna(0.0)
    out['elo_vs_opp_roll5_h'] = out['elo_vs_opp_roll5_h'].fillna(0.0)
    out['elo_vs_opp_roll5_a'] = out['elo_vs_opp_roll5_a'].fillna(0.0)

    # ------- Diffs & interactions (incl. Elo diffs you asked for) -------
    out["rest_diff"]          = out["rest_h"] - out["rest_a"]
    out["rw_pct_diff"]        = out["rw_pct_h"] - out["rw_pct_a"]
    out["streak_diff"]        = out["streak_h"] - out["streak_a"]
    out["spdiff_diff"]        = out["spdiff_h"] - out["spdiff_a"]
    out["off_eff_diff"]       = out["off_eff_h"] - out["off_eff_a"]
    out["def_eff_diff"]       = out["def_eff_h"] - out["def_eff_a"]
    out['off_def_rtg_diff']   = out['off_rtg_h'] - out['def_rtg_a']
    out['def_off_rtg_diff']   = out['def_rtg_h'] - out['off_rtg_a']
    out["net_off_diff"]       = out["off_eff_h"] - out["def_eff_a"]
    out["net_def_diff"]       = out["def_eff_h"] - out["off_eff_a"]
    out["hvenue_winpct_diff"] = out["home_win_pct_h"] - out["away_win_pct_a"]
    out["hvenue_pdiff_diff"]  = out["home_point_diff_h"] - out["away_point_diff_a"]
    out["b2b_home"] = (out["rest_h"] == 0).astype(int)
    out["b2b_away"] = (out["rest_a"] == 0).astype(int)

    # Elo diffs (these four are guaranteed present via the Elo block above, else NaN)
    out["elo_pre_ma5_diff"]    = out.get("elo_pre_ma5_h", np.nan)    - out.get("elo_pre_ma5_a", np.nan)
    out["elo_pre_slope5_diff"] = out.get("elo_pre_slope5_h", np.nan) - out.get("elo_pre_slope5_a", np.nan)
    out["elo_momentum_diff"]   = out.get("elo_momentum_h", np.nan)   - out.get("elo_momentum_a", np.nan)
    out["elo_volatility_diff"] = out.get("elo_volatility_h", np.nan) - out.get("elo_volatility_a", np.nan)
    out['elo_vs_opp_roll5_diff'] = out.get('elo_vs_opp_roll5_h', np.nan) - out.get('elo_vs_opp_roll5_a', np.nan)

    out["momentum_index"] = (
        0.4 * out["rw_pct_diff"] +
        0.3 * out["streak_diff"]/5 +
        0.3 * np.tanh(out["spdiff_diff"]/20)
    )
    if "elo_diff" in out.columns:
        out["elo_x_momentum"] = out["elo_diff"] * out["momentum_index"]
        out["elo_x_rest"]     = out["elo_diff"] * out["rest_diff"]
    out["off_def_product"] = out["off_eff_diff"] * out["def_eff_diff"]
    out["venue_x_streak"]  = out["hvenue_winpct_diff"] * out["streak_diff"]
    out['elo_vs_opp_x_streak'] = out['elo_vs_opp_roll5_diff'] * out['streak_diff']
    out['elo_vs_opp_x_winpct'] = out['elo_vs_opp_roll5_diff'] * out['rw_pct_diff']
    out['elo_vs_opp_x_PLUS_MINUS'] = out['elo_vs_opp_roll5_diff'] * out['PLUS_MINUS_roll5_diff']

    return out


def _fit_atk_def_ridge(gsub: pd.DataFrame,
                       teams: list[str],
                       lam: float = 50.0,
                       constraint_w: float = 100.0):

    T = len(teams)
    idx = {t:i for i,t in enumerate(teams)}
    G = len(gsub)

    # Design: variables = [atk_0..atk_{T-1}, def_0..def_{T-1}, hca]
    P = 2*T + 1
    rows = 2*G + 2  # 2 eqns per game + 2 constraints

    X = np.zeros((rows, P), dtype=float)
    y = np.zeros(rows, dtype=float)

    r = 0
    # Per-game equations
    for h, a, ph, pa in zip(gsub["team_home"], gsub["team_away"], gsub["score_home"], gsub["score_away"]):
        ih, ia = idx[h], idx[a]

        # home points: ph = atk[h] - def[a] + hca
        X[r, ih] = 1.0                 # atk[h]
        X[r, T + ia] = -1.0            # -def[a]
        X[r, 2*T] = 1.0                # +hca
        y[r] = float(ph); r += 1

        # away points: pa = atk[a] - def[h] - hca
        X[r, ia] = 1.0                 # atk[a]
        X[r, T + ih] = -1.0            # -def[h]
        X[r, 2*T] = -1.0               # -hca
        y[r] = float(pa); r += 1

    # Soft constraints: mean(atk)=0 and mean(def)=0
    # weight them so they behave like strong pseudo-observations
    X[r, :T] = constraint_w / np.sqrt(T)
    y[r] = 0.0; r += 1

    X[r, T:2*T] = constraint_w / np.sqrt(T)
    y[r] = 0.0; r += 1

    # Ridge solve: (X^T X + lam I)x = X^T y
    XtX = X.T @ X
    Xty = X.T @ y
    XtX += lam * np.eye(P)

    coef = np.linalg.solve(XtX, Xty)

    atk = coef[:T]
    deff = coef[T:2*T]
    hca = float(coef[2*T])

    # enforce exact mean-zero post-hoc (numerical nicety)
    atk -= atk.mean()
    deff -= deff.mean()

    return atk, deff, hca, idx

# ----- main feature builder: leak-free pre-game ratings per game -----
def add_sched_adjust_features(
    games: pd.DataFrame,
    lam: float = 50.0,
    constraint_w: float = 100.0,
    season_col: str = "sid",
) -> pd.DataFrame:

    df = games.copy()
    if season_col not in df.columns:
        raise ValueError("Need season id column (sid).")

    # normalize date
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    # stable ordering (no leakage on same day — we exclude same-day games entirely)
    df = df.sort_values([season_col, "date"]).reset_index(drop=True)

    # allocate outputs
    df["atk_pre_home"] = np.nan
    df["def_pre_home"] = np.nan
    df["atk_pre_away"] = np.nan
    df["def_pre_away"] = np.nan
    df["hca_est_pre"]  = np.nan

    # process each season independently
    for sid, g_season in df.groupby(season_col, sort=False):
        # teams present this season
        teams = pd.Index(
            pd.unique(pd.concat([g_season["team_home"], g_season["team_away"]], ignore_index=True)),
            name="team"
        ).tolist()

        # iterate over unique dates in ascending order
        dates = np.sort(g_season["date"].unique())

        # For each game day, fit ratings on games strictly before that day, then assign to all games of that day
        for d in dates:
            hist = g_season[g_season["date"] < d]
            if len(hist) < max(10, len(teams)):  # too early in season, skip (leave NaNs)
                continue

            atk, deff, hca, idx = _fit_atk_def_ridge(hist, teams, lam=lam, constraint_w=constraint_w)
            # map to dicts for quick lookup
            atk_d = {t: atk[i] for t,i in idx.items()}
            def_d = {t: deff[i] for t,i in idx.items()}

            mask_today = (df[season_col] == sid) & (df["date"] == d)
            # fill all games on that calendar day (pre-game estimates)
            df.loc[mask_today, "atk_pre_home"] = df.loc[mask_today, "team_home"].map(atk_d).astype(float)
            df.loc[mask_today, "def_pre_home"] = df.loc[mask_today, "team_home"].map(def_d).astype(float)
            df.loc[mask_today, "atk_pre_away"] = df.loc[mask_today, "team_away"].map(atk_d).astype(float)
            df.loc[mask_today, "def_pre_away"] = df.loc[mask_today, "team_away"].map(def_d).astype(float)
            df.loc[mask_today, "hca_est_pre"]  = hca

    # engineered diffs (home - away)
    df["atk_diff"]            = df["atk_pre_home"] - df["atk_pre_away"]
    df["def_diff"]            = df["def_pre_home"] - df["def_pre_away"]
    df["atk_minus_oppdef"]    = df["atk_pre_home"] - df["def_pre_away"]
    df["def_minus_oppatk"]    = df["def_pre_home"] - df["atk_pre_away"]

    return df



games_feat = add_features(games_df)
games_feat = add_sched_adjust_features(games_feat)
games_feat = games_feat.dropna()
games_feat


/var/folders/vc/9pky90h50tjdhdw9xnj_zsbc0000gn/T/ipykernel_54010/3110720185.py:157: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  logs = logs.groupby(["team","sid"], group_keys=False).apply(_streak_from_prev)
/var/folders/vc/9pky90h50tjdhdw9xnj_zsbc0000gn/T/ipykernel_54010/3110720185.py:185: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  logs = logs.groupby(["team","sid"], group_keys=False).apply(_elo_roll)
/var/folde

,sid,team_home,date,MIN_home,score_home,FGM_home,FGA_home,FG_PCT_home,FG3M_home,FG3A_home,...,elo_vs_opp_x_PLUS_MINUS,atk_pre_home,def_pre_home,atk_pre_away,def_pre_away,hca_est_pre,atk_diff,def_diff,atk_minus_oppdef,def_minus_oppatk
44,22001,PHX,2001-11-04,240,100,41,81,0.506,1,7,...,100.130967,0.154003,-0.506988,-0.284133,0.395589,0.489983,0.438135,-0.902577,-0.241586,-0.222855
45,22001,TOR,2001-11-04,240,113,37,76,0.487,6,15,...,198.958641,-0.030573,-0.259023,0.430312,0.142993,0.489983,-0.460885,-0.402016,-0.173566,-0.689335
47,22001,LAL,2001-11-04,240,100,35,68,0.515,2,11,...,257.882486,0.566321,0.119672,0.268092,-0.161084,0.489983,0.298229,0.280756,0.727404,-0.148420
49,22001,DET,2001-11-04,241,100,37,80,0.463,9,23,...,22.269377,-0.173624,0.507527,-0.157780,0.575967,0.489983,-0.015845,-0.068440,-0.749592,0.665307
50,22001,GSW,2001-11-04,241,96,35,91,0.385,4,10,...,36.127405,0.163076,-0.335211,-0.391568,-0.024879,0.489983,0.554644,-0.310333,0.187955,0.056356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28933,22025,TOR,2025-11-04,241,128,44,91,0.484,17,38,...,552.360748,0.771735,-0.868387,1.025689,-0.501629,0.777029,-0.253954,-0.366758,1.273364,-1.894076
28934,22025,CHI,2025-11-04,240,113,42,89,0.472,13,34,...,171.564587,-1.184662,1.705085,-0.874954,1.621152,0.777029,-0.309708,0.083933,-2.805814,2.580040
28935,22025,LAC,2025-11-04,242,107,33,80,0.413,18,43,...,2038.881132,-2.205860,2.044057,1.048258,0.366034,0.777029,-3.254118,1.678024,-2.571894,0.995800
28936,22025,GSW,2025-11-04,240,118,41,83,0.494,19,42,...,270.507079,0.445750,-0.179876,0.588531,-0.894624,0.777029,-0.142780,0.714748,1.340374,-0.768407


In [ ]:
ID_COLS = ["sid","date","team_home","team_away"]
TARGET  = 'margin'

FEATURE_COLS = [
    # Elo
    "elo_diff", 'elo_vs_opp_roll5_diff',
    "elo_pre_ma5_diff", "elo_pre_slope5_diff", "elo_momentum_diff", "elo_volatility_diff",
    "elo_x_momentum", "elo_x_rest", 'elo_vs_opp_x_streak', 'elo_vs_opp_x_winpct', 'elo_vs_opp_x_PLUS_MINUS',

    # Schedule Adjusted
    "atk_diff", "def_diff",
    "atk_minus_oppdef", "def_minus_oppatk",
    "atk_pre_home","def_pre_home","atk_pre_away","def_pre_away","hca_est_pre",

    # possesion stats
    'off_rtg_roll3_diff', 'off_rtg_roll5_diff',
    'def_rtg_roll3_diff', 'def_rtg_roll5_diff',

    # Context / venue / rest
    "rest_diff", "rw_pct_diff", "streak_diff", "spdiff_diff",
    "hvenue_winpct_diff", "hvenue_pdiff_diff",
    "b2b_home", "b2b_away",
    "momentum_index",

    # Four factors
    'ff_prev_diff', 'ff_roll3_diff', 'ff_roll5_diff',

    # Prev (lag-1) diffs
    'PLUS_MINUS_prev_diff','off_rtg_prev_diff', 'def_rtg_prev_diff' ,"off_eff_prev_diff", 'def_eff_prev_diff',
    'pace_prev_diff', "efg_prev_diff", "tov_rate_prev_diff", "reb_rate_prev_diff", "ft_eff_prev_diff", 'ast_tov_ratio_prev_diff',

    # Rolling 3 & 5 (shifted)
    'pace_roll3_diff', 'pace_roll5_diff',
    'ast_tov_ratio_roll3_diff', 'ast_tov_ratio_roll5_diff',
    'PLUS_MINUS_roll3_diff', 'PLUS_MINUS_roll5_diff',
    'fgp_roll3_diff', 'fgp_roll5_diff',
    'tpp_roll3_diff', 'tpp_roll5_diff',
    "off_eff_roll3_diff", "off_eff_roll5_diff",
    'def_eff_roll3_diff', 'def_eff_roll5_diff',
    "efg_roll3_diff", "efg_roll5_diff",
    "tov_rate_roll3_diff", "tov_rate_roll5_diff",
    "reb_rate_roll3_diff", "reb_rate_roll5_diff",
    "ft_eff_roll3_diff", "ft_eff_roll5_diff",
]


ALL = FEATURE_COLS + ID_COLS + ['margin']
games_f = games_feat[ALL]
games_f

,elo_diff,elo_vs_opp_roll5_diff,elo_pre_ma5_diff,elo_pre_slope5_diff,elo_momentum_diff,elo_volatility_diff,elo_x_momentum,elo_x_rest,elo_vs_opp_x_streak,elo_vs_opp_x_winpct,...,tov_rate_roll5_diff,reb_rate_roll3_diff,reb_rate_roll5_diff,ft_eff_roll3_diff,ft_eff_roll5_diff,sid,date,team_home,team_away,margin
44,61.033252,-9.531295,-9.859936,-4.027965,-4.961466,4.530715,-24.537882,61.033252,-0.000000,3.177098,...,2.685034,-0.041978,-0.041715,0.092657,0.094855,22001,2001-11-04,PHX,HOU,-3
45,48.960973,-14.111642,-15.668812,-10.589736,-8.985560,-0.201864,-23.861730,0.000000,14.111642,4.703881,...,0.106386,-0.019881,-0.018718,-0.168043,-0.173075,22001,2001-11-04,TOR,IND,13
47,108.416559,25.902104,22.997912,11.508550,10.832969,-14.674813,71.898310,108.416559,51.804208,17.268069,...,-1.954680,-0.077141,-0.075889,0.000938,-0.010991,22001,2001-11-04,LAL,UTA,4
49,74.039453,-16.212106,-2.208186,-0.538349,-0.626066,2.399280,-5.440093,74.039453,-0.000000,-0.000000,...,4.058700,-0.007427,-0.004823,0.145270,0.151862,22001,2001-11-04,DET,WAS,22
50,82.587019,7.024773,-1.655532,0.210157,2.223123,-2.661540,24.074198,0.000000,14.049546,0.000000,...,1.368870,0.054313,0.051884,-0.074090,-0.073662,22001,2001-11-04,GSW,POR,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28933,-52.040150,-104.619451,-136.037677,-1.298727,-2.347954,6.225181,17.975776,-52.040150,-104.619451,29.891272,...,0.061937,-0.012637,-0.011551,0.077217,0.051677,22025,2025-11-04,TOR,MIL,28
28934,191.671948,58.150120,132.053334,-3.858101,-3.094745,1.983512,-55.872563,0.000000,-116.300240,0.000000,...,-0.232334,0.042013,0.046688,0.009471,0.021761,22025,2025-11-04,CHI,PHI,2
28935,-52.253064,-176.291387,-116.791655,-5.747661,-4.714294,4.375361,51.204836,52.253064,1410.331096,88.145693,...,3.435928,0.006682,0.011362,-0.029584,-0.018227,22025,2025-11-04,LAC,OKC,-19
28936,177.977560,62.729947,120.199981,-2.098045,-3.275657,2.499068,15.279502,177.977560,-250.919788,8.961421,...,0.337703,0.021903,0.019150,0.022629,0.020475,22025,2025-11-04,GSW,PHX,11


In [73]:

FEATS = FEATURE_COLS

monos = "(" + ",".join(["1"] + ["0"]*(len(FEATS)-1)) + ")"

def time_splits(df, date_col="date", splits=(0.6, 0.2, 0.2)):
    assert abs(sum(splits) - 1.0) < 1e-9
    d = df.sort_values(date_col).reset_index(drop=True)
    cut1 = d[date_col].quantile(splits[0])
    cut2 = d[date_col].quantile(splits[0] + splits[1])
    idx_tr = np.flatnonzero(d[date_col] <  cut1)
    idx_va = np.flatnonzero((d[date_col] >= cut1) & (d[date_col] < cut2))
    idx_te = np.flatnonzero(d[date_col] >= cut2)
    return idx_tr, idx_va, idx_te

games_sorted = games_f.sort_values("date").reset_index(drop=True)
idx_tr, idx_va, idx_te = time_splits(games_sorted, "date", (0.6, 0.2, 0.2))

X = games_sorted.loc[:, FEATS]
y = games_sorted["margin"]

X_tr, y_tr = X.iloc[idx_tr], y.iloc[idx_tr]
X_va, y_va = X.iloc[idx_va], y.iloc[idx_va]
X_te, y_te = X.iloc[idx_te], y.iloc[idx_te]

X_va = X_va.reindex(columns=X_tr.columns)
X_te = X_te.reindex(columns=X_tr.columns)

date_series = games_sorted["date"].dt.normalize()

groups_tr = date_series.iloc[idx_tr].to_numpy()
groups_va = date_series.iloc[idx_va].to_numpy()
groups_te = date_series.iloc[idx_te].to_numpy()



In [79]:

# ============================================================
# NBA Spread (Margin) Stack — XGBoost + MLP tuning
# ============================================================

import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Optional, Tuple
from scipy.stats import norm, randint, uniform, loguniform
from sklearn.experimental import enable_hist_gradient_boosting  # noqa: F401
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge, ElasticNetCV, LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import make_scorer, mean_absolute_error, mean_pinball_loss
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.utils.validation import check_is_fitted
import xgboost as xgb

ABS_NORMAL_FACTOR = np.sqrt(2.0 / np.pi)

# ----------------------------
# Utilities
# ----------------------------
def mae_rmse(y_true, y_pred) -> Tuple[float, float]:
    err = y_true - y_pred
    mae = float(np.mean(np.abs(err)))
    rmse = float(np.sqrt(np.mean(err**2)))
    return mae, rmse

def gaussian_crps(y, mu, sigma):
    z = (y - mu) / sigma
    return sigma * ( z*(2*norm.cdf(z)-1) + 2*norm.pdf(z) - 1/np.sqrt(np.pi) )

def coverage(y, mu, sigma, k=1.0) -> float:
    return float(np.mean(np.abs(y - mu) <= k * sigma))

def prob_home_win_from_margin(mu, sigma):
    return 1.0 - norm.cdf((0.0 - mu) / sigma)

def fit_sigma_affine(y_true, mu_pred, sigma_pred, mode="constrained"):
    """
    Calibrate sigma on validation. Modes:
      - "scale":     sigma' = a * sigma        (no intercept)
      - "constrained": sigma' = a * sigma + b  with a >= 0.2 (keeps relative variation)
    Returns (a, b).
    """
    r = np.abs(y_true - mu_pred)
    s = np.asarray(sigma_pred)

    if mode == "scale":
        denom = np.dot(s, s) + 1e-12
        a = float(np.dot(s, r) / denom)
        a = max(a, 1e-6)
        return a, 0.0

    X = np.column_stack([s, np.ones_like(s)])
    a_b, *_ = np.linalg.lstsq(X, r, rcond=None)
    a, b = map(float, a_b)
    a = max(a, 0.2)
    b = max(b, 0.0)
    return a, b

# ----------------------------
# Custom Quantile-XGB wrapper
# ----------------------------
class QuantileXGBRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=0.5, hess_eps=1e-6, **xgb_params):
        self.alpha = alpha
        self.hess_eps = hess_eps
        self.xgb_params = xgb_params
        self._booster = None
        self._num_boost_round = None

    def _pinball_obj(self, preds: np.ndarray, dtrain: xgb.DMatrix):
        y = dtrain.get_label()
        u = y - preds
        grad = np.where(u > 0.0, -self.alpha, 1.0 - self.alpha)
        hess = np.full_like(grad, self.hess_eps)
        return grad, hess

    def get_params(self, deep=True):
        return {"alpha": self.alpha, "hess_eps": self.hess_eps, **self.xgb_params}

    def set_params(self, **params):
        if "alpha" in params:
            self.alpha = params.pop("alpha")
        if "hess_eps" in params:
            self.hess_eps = params.pop("hess_eps")
        self.xgb_params.update(params)
        return self

    def _to_train_params(self):
        params = self.xgb_params.copy()
        self._num_boost_round = int(params.pop("n_estimators", 800))
        if "learning_rate" in params:
            params["eta"] = params.pop("learning_rate")
        if "n_jobs" in params:
            params["nthread"] = params.pop("n_jobs")
        if "random_state" in params:
            params["seed"] = params.pop("random_state")
        params.setdefault("tree_method", "hist")
        params.setdefault("objective", "reg:squarederror")
        return params

    def fit(self, X, y):
        dtrain = xgb.DMatrix(X, label=y)
        params = self._to_train_params()
        self._booster = xgb.train(params=params, dtrain=dtrain,
                                  num_boost_round=self._num_boost_round,
                                  obj=self._pinball_obj, verbose_eval=False)
        return self

    def predict(self, X):
        return self._booster.predict(xgb.DMatrix(X))


# ----------------------------
# Time-aware stacking regressor
# ----------------------------
class TwoStageStackTSRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, base_estimators, meta_estimator, n_splits=6, group_gap=1, verbose=False):
        self.base_estimators = base_estimators
        self.meta_estimator = meta_estimator
        self.n_splits = n_splits
        self.group_gap = group_gap
        self.verbose = verbose

    def _safe_slice(self, X, idx):
        return X.iloc[idx] if hasattr(X, "iloc") else X[idx]

    def fit(self, X, y, groups=None):
        if groups is None:
            raise ValueError("TwoStageStackTSRegressor requires 'groups' for time-aware CV")

        y_arr = np.asarray(y, dtype=float)
        groups_arr = np.asarray(groups)
        if groups_arr.shape[0] != y_arr.shape[0]:
            raise ValueError("groups must align with y")

        splitter = PurgedGroupTimeSeriesSplit(n_splits=self.n_splits, group_gap=self.group_gap)
        dummy_X = np.zeros((len(y_arr), 1))
        splits = list(splitter.split(dummy_X, y_arr, groups_arr))

        oof_cols = []
        self.base_fitted_ = []
        for name, est in self.base_estimators:
            oof = np.full(len(y_arr), np.nan, dtype=float)
            for fold_idx, (tr_idx, va_idx) in enumerate(splits):
                model = clone(est)
                model.fit(self._safe_slice(X, tr_idx), y_arr[tr_idx])
                oof[va_idx] = model.predict(self._safe_slice(X, va_idx))
            mask = ~np.isnan(oof)
            if not np.any(mask):
                raise RuntimeError(f"No validation coverage for base estimator '{name}'")
            oof_cols.append(oof)
            fitted = clone(est)
            fitted.fit(X, y_arr)
            self.base_fitted_.append((name, fitted))

        Z = np.column_stack(oof_cols)
        mask_all = np.all(~np.isnan(Z), axis=1)
        if not np.any(mask_all):
            raise RuntimeError("No overlapping OOF coverage across base estimators")
        self.meta_ = clone(self.meta_estimator).fit(Z[mask_all], y_arr[mask_all])
        self.mask_ = mask_all
        self.base_names_ = [name for name, _ in self.base_estimators]
        self.n_features_in_ = Z.shape[1]
        return self

    def predict(self, X):
        check_is_fitted(self, ["meta_", "base_fitted_"])
        preds = [est.predict(X) for _, est in self.base_fitted_]
        Z = np.column_stack(preds)
        return self.meta_.predict(Z)

# ----------------------------
# Small search spaces & tuners (XGB + MLP + HGB)
# ----------------------------
pinball50 = make_scorer(mean_pinball_loss, alpha=0.50, greater_is_better=False)
neg_mae   = make_scorer(mean_absolute_error, greater_is_better=False)

def small_space_xgb_mu():
    return {
        "n_estimators": randint(600, 1400),
        "learning_rate": loguniform(0.015, 0.08),
        "max_depth": randint(4, 9),
        "min_child_weight": loguniform(0.7, 6.0),
        "subsample": uniform(0.7, 0.25),
        "colsample_bytree": uniform(0.7, 0.25),
        "gamma": loguniform(1e-3, 3e-1),
        "reg_alpha": loguniform(1e-5, 5e-2),
        "reg_lambda": loguniform(1e-2, 5.0)
    }

def small_space_xgb_quantile():
    return {
        "n_estimators": randint(600, 1400),
        "learning_rate": loguniform(0.015, 0.08),
        "max_depth": randint(4, 9),
        "min_child_weight": loguniform(0.7, 6.0),
        "subsample": uniform(0.7, 0.25),
        "colsample_bytree": uniform(0.7, 0.25),
        "gamma": loguniform(1e-3, 3e-1),
        "reg_alpha": loguniform(1e-5, 5e-2),
        "reg_lambda": loguniform(1e-2, 5.0)
    }

def small_space_mlp():
    return {
        "mlp__hidden_layer_sizes": [(128, 64), (192, 96), (128, 64, 32), (256, 128)],
        "mlp__activation": ["relu", "tanh"],
        "mlp__alpha": loguniform(1e-5, 5e-2),
        "mlp__learning_rate_init": loguniform(1e-4, 2e-2),
        "mlp__batch_size": [64, 128, 256]
    }

def small_space_hgb():
    return {
        "learning_rate": loguniform(0.01, 0.2),
        "max_depth": randint(3, 9),
        "max_leaf_nodes": randint(32, 180),
        "min_samples_leaf": randint(20, 140),
        "l2_regularization": loguniform(1e-6, 1e-1)
    }

def small_space_etr():
    return {
        "n_estimators": randint(300, 900),
        "max_depth": randint(4, 16),
        "min_samples_split": randint(2, 20),
        "min_samples_leaf": randint(1, 10),
        "max_features": uniform(0.4, 0.6)
    }

def tune_mlp(X, y, n_iter=40, n_splits=6, random_state=42):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    pipe = Pipeline(steps=[
        ("scaler", StandardScaler(with_mean=True, with_std=True)),
        ("mlp", MLPRegressor(solver="adam", early_stopping=True, max_iter=700,
                             random_state=random_state))
    ])
    rs = RandomizedSearchCV(
        pipe, param_distributions=small_space_mlp(), n_iter=n_iter,
        scoring=neg_mae, cv=tscv, verbose=1, random_state=random_state, n_jobs=-1
    )
    rs.fit(X, y)
    print("Best MLP params:", rs.best_params_)
    print("CV MAE (MLP-only):", -rs.best_score_)
    best = {k.split("mlp__")[1]: v for k, v in rs.best_params_.items() if k.startswith("mlp__")}
    best.update(dict(solver="adam", early_stopping=True, max_iter=700, random_state=random_state))
    return best

def tune_xgb_mu(X, y, n_iter=80, n_splits=6, random_state=42):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    base = xgb.XGBRegressor(objective="reg:squarederror",
                            random_state=random_state, n_jobs=-1, tree_method="hist")
    rs = RandomizedSearchCV(
        base, param_distributions=small_space_xgb_mu(), n_iter=n_iter,
        scoring=neg_mae, cv=tscv, verbose=1, random_state=random_state, n_jobs=-1
    )
    rs.fit(X, y)
    print("Best μ params (XGB):", rs.best_params_)
    print("CV MAE:", -rs.best_score_)
    return rs.best_params_

def tune_xgb_quantile(X, y, alpha, n_iter=80, n_splits=6, random_state=42):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    search_seed = random_state + int(round(alpha * 100))
    base = QuantileXGBRegressor(alpha=alpha, random_state=search_seed, n_jobs=-1, tree_method="hist")
    rs = RandomizedSearchCV(
        base, param_distributions=small_space_xgb_quantile(), n_iter=n_iter,
        scoring=make_scorer(mean_pinball_loss, alpha=alpha, greater_is_better=False),
        cv=tscv, verbose=1, random_state=search_seed, n_jobs=-1
    )
    rs.fit(X, y)
    print(f"Best quantile({alpha:.2f}) params (XGB):", rs.best_params_)
    print(f"CV pinball({alpha:.2f}):", -rs.best_score_)
    return {k: v for k, v in rs.best_params_.items() if k not in ("alpha", "hess_eps")}

def tune_hgb_mu(X, y, n_iter=40, n_splits=6, random_state=42):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    base = HistGradientBoostingRegressor(loss="squared_error", random_state=random_state)
    rs = RandomizedSearchCV(
        base, param_distributions=small_space_hgb(), n_iter=n_iter,
        scoring=neg_mae, cv=tscv, verbose=1, random_state=random_state, n_jobs=-1
    )
    rs.fit(X, y)
    print("Best μ params (HGB):", rs.best_params_)
    print("CV MAE (HGB):", -rs.best_score_)
    return rs.best_params_

def tune_etr_mu(X, y, n_iter=40, n_splits=6, random_state=42):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    base = ExtraTreesRegressor(random_state=random_state, n_jobs=-1)
    rs = RandomizedSearchCV(
        base, param_distributions=small_space_etr(), n_iter=n_iter,
        scoring=neg_mae, cv=tscv, verbose=1, random_state=random_state, n_jobs=-1
    )
    rs.fit(X, y)
    print("Best μ params (ETR):", rs.best_params_)
    print("CV MAE (ETR):", -rs.best_score_)
    return rs.best_params_

# ----------------------------
# Model
# ----------------------------
@dataclass
class SpreadStackXGB:
    xgb_mu_params: Optional[dict] = None
    xgb_q50_params: Optional[dict] = None
    xgb_q16_params: Optional[dict] = None
    xgb_q84_params: Optional[dict] = None
    mlp_params: Optional[dict] = None
    hgb_params: Optional[dict] = None
    etr_params: Optional[dict] = None
    sigma_model_params: Optional[dict] = None
    tune_mu_iter: int = 80
    tune_q_iter: int = 80
    tune_mlp_iter: int = 40
    tune_hgb_iter: int = 40
    tscv_splits: int = 6
    random_state: int = 42
    sigma_min_range: float = 0.3
    sigma_min_rel: float = 0.03
    stack_splits: int = 6
    stack_gap: int = 1
    stack_verbose: bool = False
    sigma_blend_grid: Tuple[float, float, int] = (0.0, 1.0, 11)

    def __post_init__(self):
        self.mu_stack_: Optional[TwoStageStackTSRegressor] = None
        self.q16_: Optional[QuantileXGBRegressor] = None
        self.q50_: Optional[QuantileXGBRegressor] = None
        self.q84_: Optional[QuantileXGBRegressor] = None
        self.sigma_model_: Optional[xgb.XGBRegressor] = None
        self.fallback_sigma_: bool = False
        self.sigma_source_: str = "quantiles"
        self.sigma_blend_weight_: float = 1.0
        self.sigma_cal_: Tuple[float, float] = (1.0, 0.0)
        self.mu_calibration_: Tuple[float, float] = (1.0, 0.0)
        self._va_diag_: Optional[dict] = None

    def fit(self, X_tr: pd.DataFrame, y_tr: np.ndarray,
                  X_va: pd.DataFrame, y_va: np.ndarray,
                  *, groups_tr: Optional[np.ndarray] = None,
                  groups_va: Optional[np.ndarray] = None):
        if groups_tr is None:
            raise ValueError("SpreadStackXGB.fit requires groups_tr for time-aware stacking")

        # --- Tune params if not provided ---
        if self.xgb_mu_params is None:
            self.xgb_mu_params = tune_xgb_mu(X_tr, y_tr,
                                             n_iter=self.tune_mu_iter,
                                             n_splits=self.tscv_splits,
                                             random_state=self.random_state)
            self.xgb_mu_params.update(dict(objective="reg:squarederror",
                                           random_state=self.random_state, n_jobs=-1, tree_method="hist"))
        if self.xgb_q50_params is None:
            q50_best = tune_xgb_quantile(X_tr, y_tr, 0.5,
                                           n_iter=self.tune_q_iter,
                                           n_splits=self.tscv_splits,
                                           random_state=self.random_state)
            self.xgb_q50_params = {**q50_best, "random_state": self.random_state, "n_jobs": -1, "tree_method": "hist"}

        if self.xgb_q16_params is None:
            q16_best = tune_xgb_quantile(X_tr, y_tr, 0.16,
                                           n_iter=self.tune_q_iter,
                                           n_splits=self.tscv_splits,
                                           random_state=self.random_state)
            self.xgb_q16_params = {**q16_best, "random_state": self.random_state, "n_jobs": -1, "tree_method": "hist"}

        if self.xgb_q84_params is None:
            q84_best = tune_xgb_quantile(X_tr, y_tr, 0.84,
                                           n_iter=self.tune_q_iter,
                                           n_splits=self.tscv_splits,
                                           random_state=self.random_state)
            self.xgb_q84_params = {**q84_best, "random_state": self.random_state, "n_jobs": -1, "tree_method": "hist"}

        if self.mlp_params is None:
            self.mlp_params = tune_mlp(X_tr, y_tr,
                                       n_iter=self.tune_mlp_iter,
                                       n_splits=self.tscv_splits,
                                       random_state=self.random_state)

        if self.hgb_params is None:
            self.hgb_params = tune_hgb_mu(X_tr, y_tr,
                                          n_iter=self.tune_hgb_iter,
                                          n_splits=self.tscv_splits,
                                          random_state=self.random_state)

        if self.etr_params is None:
            self.etr_params = tune_etr_mu(X_tr, y_tr,
                                          n_iter=self.tune_hgb_iter,
                                          n_splits=self.tscv_splits,
                                          random_state=self.random_state)

        # --- μ via time-aware stacking (XGB + tuned MLP + HGB) ---
        xgb_mu = xgb.XGBRegressor(**self.xgb_mu_params)
        mlp_mu = make_pipeline(StandardScaler(with_mean=True, with_std=True),
                               MLPRegressor(**self.mlp_params))
        hgb_mu = HistGradientBoostingRegressor(**self.hgb_params)
        etr_mu = ExtraTreesRegressor(**self.etr_params)
        meta = make_pipeline(
            StandardScaler(with_mean=True, with_std=True),
            ElasticNetCV(l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 1.0],
                         n_alphas=40,
                         cv=5,
                         random_state=self.random_state,
                         n_jobs=-1,
                         selection="random")
        )

        base_estimators = [
            ("xgb_mu", xgb_mu),
            ("mlp_mu", mlp_mu),
            ("hgb_mu", hgb_mu),
            ("etr_mu", etr_mu)
        ]

        self.mu_stack_ = TwoStageStackTSRegressor(
            base_estimators=base_estimators,
            meta_estimator=meta,
            n_splits=self.stack_splits,
            group_gap=self.stack_gap,
            verbose=self.stack_verbose
        )
        self.mu_stack_.fit(X_tr, y_tr, groups=groups_tr)

        # --- σ via quantiles (q16/q50/q84) and residual model ---
        self.q50_ = QuantileXGBRegressor(alpha=0.50, **self.xgb_q50_params).fit(X_tr, y_tr)
        self.q16_ = QuantileXGBRegressor(alpha=0.16, **self.xgb_q16_params).fit(X_tr, y_tr)
        self.q84_ = QuantileXGBRegressor(alpha=0.84, **self.xgb_q84_params).fit(X_tr, y_tr)

        mu_va_stack = self.mu_stack_.predict(X_va)
        q50_va = self.q50_.predict(X_va)
        mu_va_blend = 0.7 * mu_va_stack + 0.3 * q50_va

        lr_mu = LinearRegression()
        lr_mu.fit(mu_va_blend.reshape(-1, 1), y_va)
        slope = float(lr_mu.coef_[0])
        intercept = float(lr_mu.intercept_)
        self.mu_calibration_ = (slope, intercept)
        mu_va_cal = slope * mu_va_blend + intercept

        sigma_quant_va = (self.q84_.predict(X_va) - self.q16_.predict(X_va)) / 2.0
        sigma_quant_va = np.asarray(sigma_quant_va, dtype=float)

        mu_tr_stack = self.mu_stack_.predict(X_tr)
        q50_tr = self.q50_.predict(X_tr)
        mu_tr_blend = 0.7 * mu_tr_stack + 0.3 * q50_tr
        mu_tr_cal = slope * mu_tr_blend + intercept
        resid_tr = np.abs(y_tr - mu_tr_cal)
        target_sigma_tr = resid_tr / ABS_NORMAL_FACTOR

        sigma_params = dict(self.sigma_model_params) if self.sigma_model_params is not None else {
            "objective": "reg:squarederror",
            "random_state": self.random_state,
            "n_estimators": 800,
            "learning_rate": 0.05,
            "max_depth": 6,
            "subsample": 0.85,
            "colsample_bytree": 0.85,
            "min_child_weight": 1.0,
            "n_jobs": -1,
            "tree_method": "hist",
        }

        self.sigma_model_ = xgb.XGBRegressor(**sigma_params)
        self.sigma_model_.fit(X_tr, target_sigma_tr)
        sigma_resid_va = np.asarray(self.sigma_model_.predict(X_va), dtype=float)

        sigma_quant_va = np.where(sigma_quant_va <= 0, 1e-6, sigma_quant_va)
        sigma_resid_va = np.where(sigma_resid_va <= 0, 1e-6, sigma_resid_va)

        sigma_range = float(np.ptp(sigma_quant_va)) if sigma_quant_va.size else 0.0
        sigma_mean = float(np.mean(sigma_quant_va)) if sigma_quant_va.size else 0.0
        rel_range = sigma_range / max(abs(sigma_mean), 1e-6) if sigma_quant_va.size else 0.0
        use_fallback = (not np.isfinite(sigma_range)) or (sigma_range < self.sigma_min_range and rel_range < self.sigma_min_rel)

        if use_fallback:
            self.fallback_sigma_ = True
            self.sigma_blend_weight_ = 0.0
            sigma_va = sigma_resid_va
        else:
            blend_min, blend_max, blend_steps = self.sigma_blend_grid
            weights = np.linspace(blend_min, blend_max, int(blend_steps)) if blend_steps > 1 else np.array([blend_min])
            best_w, best_crps, best_sigma = 1.0, np.inf, sigma_quant_va
            for w in weights:
                sigma_candidate = w * sigma_quant_va + (1.0 - w) * sigma_resid_va
                sigma_candidate = np.where(sigma_candidate <= 0, 1e-6, sigma_candidate)
                crps = float(np.mean(gaussian_crps(y_va, mu_va_cal, sigma_candidate)))
                if crps < best_crps:
                    best_crps = crps
                    best_w = float(w)
                    best_sigma = sigma_candidate
            self.fallback_sigma_ = False
            self.sigma_blend_weight_ = best_w
            sigma_va = best_sigma

        sigma_va = np.where(sigma_va <= 0, 1e-6, sigma_va)

        a, b = fit_sigma_affine(y_va, mu_va_cal, sigma_va, mode="constrained")
        self.sigma_cal_ = (a, b)

        if self.fallback_sigma_:
            self.sigma_source_ = "residual"
        else:
            if self.sigma_blend_weight_ >= 0.99:
                self.sigma_source_ = "quantiles"
            elif self.sigma_blend_weight_ <= 0.01:
                self.sigma_source_ = "residual"
            else:
                self.sigma_source_ = "blend"

        self._va_diag_ = {
            "mu": mu_va_cal.tolist(),
            "sigma_quant": sigma_quant_va.tolist(),
            "sigma_resid": sigma_resid_va.tolist(),
            "sigma_final": sigma_va.tolist(),
            "a": a,
            "b": b,
            "y": y_va.tolist(),
            "sigma_source": self.sigma_source_,
            "sigma_range_quant": float(np.ptp(sigma_quant_va)),
            "sigma_mean_quant": float(np.mean(sigma_quant_va)),
            "blend_weight_quantile": self.sigma_blend_weight_,
            "fallback_used": self.fallback_sigma_,
            "mu_calibration": {"slope": slope, "intercept": intercept}
        }
        return self

    def _predict_components(self, X: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        mu_stack = self.mu_stack_.predict(X)
        q50 = self.q50_.predict(X)
        mu = 0.7 * mu_stack + 0.3 * q50

        sigma_quant = (self.q84_.predict(X) - self.q16_.predict(X)) / 2.0
        sigma_quant = np.asarray(sigma_quant, dtype=float)
        sigma_quant = np.where(sigma_quant <= 0, 1e-6, sigma_quant)

        sigma_resid = np.asarray(self.sigma_model_.predict(X), dtype=float) if self.sigma_model_ is not None else sigma_quant
        sigma_resid = np.where(sigma_resid <= 0, 1e-6, sigma_resid)

        if self.fallback_sigma_:
            sigma = sigma_resid
        else:
            sigma = self.sigma_blend_weight_ * sigma_quant + (1.0 - self.sigma_blend_weight_) * sigma_resid

        slope, intercept = self.mu_calibration_
        mu = slope * mu + intercept

        sigma = np.where(sigma <= 0, 1e-6, sigma)
        a, b = self.sigma_cal_
        sigma_cal = a * sigma + b
        sigma_cal = np.where(sigma_cal <= 0, 1e-6, sigma_cal)
        return mu, sigma_cal

    def predict_frame(self, X: pd.DataFrame) -> pd.DataFrame:
        mu, sigma = self._predict_components(X)
        frame = pd.DataFrame({
            "mu_margin": mu,
            "sigma_margin": sigma,
            "p_home_win": prob_home_win_from_margin(mu, sigma),
            "model_spread_home": -mu
        }, index=X.index)
        frame["sigma_source"] = self.sigma_source_
        frame["sigma_weight_quantile"] = self.sigma_blend_weight_
        return frame

# ----------------------------
# TRAIN & EVALUATE
# ----------------------------
spread_stack = SpreadStackXGB().fit(X_tr, y_tr, X_va, y_va, groups_tr=groups_tr, groups_va=groups_va)

pred_va = spread_stack.predict_frame(X_va)
mu_va, sigma_va = pred_va["mu_margin"].values, pred_va["sigma_margin"].values

mae, rmse = mae_rmse(y_va, mu_va)
cov68 = coverage(y_va, mu_va, sigma_va, k=1.0)
cov95 = coverage(y_va, mu_va, sigma_va, k=2.0)
crps = float(np.mean(gaussian_crps(y_va, mu_va, sigma_va)))

print(f"[Validation]")
print(f"MAE = {mae:.3f}   RMSE = {rmse:.3f}")
print(f"Coverage @1σ = {cov68:.3f} (target ≈ 0.683)   @2σ = {cov95:.3f} (target ≈ 0.954)")
print(f"Mean CRPS = {crps:.3f}")
bias = float(np.mean(mu_va - y_va))
med_err = float(np.median(mu_va - y_va))
print(f"Bias (mean error) = {bias:.3f} | Median error = {med_err:.3f}")
print(f"Sigma source: {spread_stack.sigma_source_} | blend weight (quantiles) = {spread_stack.sigma_blend_weight_:.3f}")
print(f"Mu calibration: slope={spread_stack.mu_calibration_[0]:.4f}, intercept={spread_stack.mu_calibration_[1]:.4f}")
if spread_stack._va_diag_ is not None:
    print("Diag preview:", {k: spread_stack._va_diag_[k] for k in ["sigma_source", "blend_weight_quantile", "fallback_used"]})
    print("Mu calib params:", spread_stack._va_diag_.get("mu_calibration"))

# Optional test predictions
try:
    pred_te = spread_stack.predict_frame(X_te)
    print("\n[Test] Head:")
    print(pred_te[["mu_margin","sigma_margin","p_home_win","model_spread_home","sigma_source","sigma_weight_quantile"]].head())
except NameError:
    pred_te = None



Fitting 6 folds for each of 80 candidates, totalling 480 fits


/Users/jamesvournakis/Desktop/Projects/nba_model/.venv/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best μ params (XGB): {'colsample_bytree': np.float64(0.7673108077373452), 'gamma': np.float64(0.008601587656840977), 'learning_rate': np.float64(0.01551254285558905), 'max_depth': 4, 'min_child_weight': np.float64(4.261133996481091), 'n_estimators': 685, 'reg_alpha': np.float64(0.00016270574291440775), 'reg_lambda': np.float64(0.02104914217006881), 'subsample': np.float64(0.9226318201849737)}
CV MAE: 9.384044329325357
Fitting 6 folds for each of 80 candidates, totalling 480 fits
Best quantile(0.50) params (XGB): {'colsample_bytree': np.float64(0.9214701248594064), 'gamma': np.float64(0.08982808891680345), 'learning_rate': np.float64(0.030539932325755536), 'max_depth': 8, 'min_child_weight': np.float64(0.7005326744482511), 'n_estimators': 1359, 'reg_alpha': np.float64(9.009866832998004e-05), 'reg_lambda': np.float64(3.2921891287053415), 'subsample': np.float64(0.7051987514668574)}
CV pinball(0.50): 5.365166266759236
Fitting 6 folds for each of 80 candidates, totalling 480 fits
Best quan

/Users/jamesvournakis/Desktop/Projects/nba_model/.venv/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best μ params (ETR): {'max_depth': 6, 'max_features': np.float64(0.754535765912945), 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 810}
CV MAE (ETR): 9.27568242460656


/Users/jamesvournakis/Desktop/Projects/nba_model/.venv/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:1622: FutureWarning: 'n_alphas' was deprecated in 1.7 and will be removed in 1.9. 'alphas' now accepts an integer value which removes the need to pass 'n_alphas'. The default value of 'alphas' will change from None to 100 in 1.9. Pass an explicit value to 'alphas' and leave 'n_alphas' to its default value to silence this warning.
  warnings.warn(


[Validation]
MAE = 9.969   RMSE = 12.767
Coverage @1σ = 0.638 (target ≈ 0.683)   @2σ = 0.920 (target ≈ 0.954)
Mean CRPS = 7.152
Bias (mean error) = -0.000 | Median error = -0.057
Sigma source: residual | blend weight (quantiles) = 0.000
Mu calibration: slope=1.3911, intercept=-0.1872
Diag preview: {'sigma_source': 'residual', 'blend_weight_quantile': 0.0, 'fallback_used': True}
Mu calib params: {'slope': 1.391073113871251, 'intercept': -0.18717800001879992}

[Test] Head:
       mu_margin  sigma_margin  p_home_win  model_spread_home sigma_source  \
22192  -4.120032     11.316211    0.357898           4.120032     residual   
22193   2.640865     12.215816    0.585578          -2.640865     residual   
22194   5.170536     11.091322    0.679456          -5.170536     residual   
22195   2.447408     11.093875    0.587302          -2.447408     residual   
22196   4.289424     11.440736    0.646142          -4.289424     residual   

       sigma_weight_quantile  
22192                   

In [84]:
(np.where(pred_te['p_home_win'] > 0.5,  1, 0) == games_feat['win_home'].iloc[idx_te]).value_counts()

win_home
True     3071
False    2477
Name: count, dtype: int64